In [19]:
import matplotlib
import tiktoken
import torch
import torch.nn as nn


GPT_CONFIG_124M = {
"vocab_size": 50257, # Vocabulary size
"context_length": 1024, # Context length
"emb_dim": 768, # Embedding dimension
"n_heads": 12, # Number of attention heads
"n_layers": 12, # Number of layers
"drop_rate": 0.1, # Dropout rate
"qkv_bias": False # Query-Key-Value bias
}

GPT Model backbones

In [2]:
class BaseGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(*[BaseTransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm = BaseLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)

        return logits   


Transformer Block backbones


In [3]:
class BaseTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        return x

class BaseLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
    
    def forward(self, x):
        return x


In [4]:
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves a"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))

batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  257],
        [6109, 1110, 6622,  257]])


In [5]:
torch.manual_seed(12423)
model = BaseGPTModel(GPT_CONFIG_124M)
logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 2.2158,  0.6168,  1.6288,  ..., -0.6449, -0.3533,  1.9425],
         [-0.7857, -0.6058,  0.4549,  ...,  1.0483,  0.7585,  1.0739],
         [-2.7803, -0.7292,  0.3345,  ...,  0.8783,  1.3939,  0.9193],
         [ 0.6966,  0.8221, -0.6616,  ..., -0.7818,  0.1671,  1.0633]],

        [[ 1.5109,  0.4453,  1.4775,  ..., -1.3766, -0.2224,  1.9125],
         [-1.4151, -0.0452, -0.4428,  ...,  0.8530,  0.5996,  1.7292],
         [-3.0778, -0.4633, -1.2628,  ...,  0.1009, -0.3442,  0.8746],
         [ 0.3622,  0.6687, -0.9696,  ..., -1.0015,  0.0678,  0.3372]]],
       grad_fn=<UnsafeViewBackward0>)


Layer Normalization

In [6]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-8
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
    
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean)/torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift


GELU Activation

In [35]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi)) *
               (x + 0.044715 * torch.pow(x, 3))))

Feed-Forward Network

In [31]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(nn.Linear(cfg["emb_dim"], 4*cfg["emb_dim"]),
                                    GELU(),
                                    nn.Linear(4*cfg["emb_dim"], cfg["emb_dim"]))
        
    def forward(self, x):
        return self.layers(x)


Transformer Block


In [24]:
#Previously implemented multihead attention
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens] # type: ignore

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec



In [33]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layernorm1 = LayerNorm(cfg["emb_dim"])
        self.layernorm2 = LayerNorm(cfg["emb_dim"])
        self.ff = FeedForward(cfg)
        self.attn = MultiHeadAttention(
                    d_in=cfg["emb_dim"],
                    d_out=cfg["emb_dim"],
                    context_length=cfg["context_length"], # type: ignore
                    num_heads=cfg["n_heads"],
                    dropout=cfg["drop_rate"],
                    qkv_bias=cfg["qkv_bias"])
        self.drop = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        shortcut = x 
        x = self.layernorm1(x)
        x = self.attn(x)
        x = self.drop(x)
        x = x + shortcut    #residual connection

        shortcut = x
        x = self.layernorm2(x)
        x = self.ff(x)
        x = self.drop(x)
        x = x + shortcut    #residual connection
        
        return x

In [36]:
torch.manual_seed(42)
x = torch.rand(2, 4, 768)
block = TransformerBlock(GPT_CONFIG_124M)
y = block(x)

print(x.shape)
print(y.shape)

torch.Size([2, 4, 768])
torch.Size([2, 4, 768])


Final GPT Model

In [38]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(*[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)

        return logits   


In [42]:
torch.manual_seed(42)
model = GPTModel(GPT_CONFIG_124M)

print(batch)
print(batch.shape) #[2,4] -> 2 input samples in the whole batch, each sample has 4 token

out = model(batch)

print(out)
print(out.shape) #[2,4,50257] -> Each token in a sample will be associated to a vector of the size of the vocab and represents 
                 # its corresponding next token with unnormalized probabilities


tensor([[6109, 3626, 6100,  257],
        [6109, 1110, 6622,  257]])
torch.Size([2, 4])
tensor([[[ 0.4206, -0.1932, -1.2388,  ..., -0.7756, -0.4191,  0.6194],
         [-0.1576,  0.3502,  0.0621,  ..., -0.5504,  0.0636, -0.3326],
         [ 0.2907,  0.3612, -0.0612,  ..., -0.2800,  0.3976,  0.5490],
         [-0.1594, -0.2894,  0.8138,  ..., -0.6677,  0.2822, -0.6924]],

        [[-0.1750,  0.0741, -0.7638,  ..., -1.2703, -0.9485,  0.3117],
         [ 0.1013,  0.4391,  0.2547,  ..., -0.6833,  0.3333, -0.8445],
         [ 0.3821,  0.4598,  0.1499,  ..., -0.2302,  0.6344,  0.9745],
         [-0.4289,  0.0022,  1.0679,  ..., -0.6732, -0.2792, -0.4485]]],
       grad_fn=<UnsafeViewBackward0>)
torch.Size([2, 4, 50257])


Postprocessing step (generate readable text)

In [53]:
def generate_text_simple(model, input_tokens, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):
        
        idx_cond = input_tokens[:, -context_size:]
        
        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)
        
        logits = logits[:, -1, :]  

        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        next_token_idx = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        input_tokens = torch.cat((input_tokens, next_token_idx), dim=1)  # (batch, n_tokens+1)
    return input_tokens


In [55]:
input = "hello, I am"
encoded = tokenizer.encode(input)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print(encoded_tensor)

model.eval()

output = generate_text_simple(model, encoded_tensor, 6, GPT_CONFIG_124M["context_length"])
print(output)

decoded = tokenizer.decode(output.squeeze(0).tolist())
print(decoded)

tensor([[31373,    11,   314,   716]])
tensor([[31373,    11,   314,   716, 21786, 42629,  3116,  8631, 46302, 14559]])
hello, I am envision hampered Serv plain786 steep
